In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class ModelNetDataset(Dataset):
    def __init__(self, root_dir, split='train', num_points=1024, single_view=True):
        """
        ModelNet数据集加载器
        Args:
            root_dir: 数据集根目录
            split: 'train' 或 'test'
            num_points: 每个点云的点数量
            single_view: 是否只使用单视角数据
        """
        self.root_dir = root_dir
        self.split = split
        self.num_points = num_points
        self.single_view = single_view
        
        # 获取所有类别
        self.categories = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.cat_to_idx = {cat: idx for idx, cat in enumerate(self.categories)}
        
        # 收集文件路径
        self.data_paths = []
        self.labels = []
        
        # 遍历所有类别文件夹
        for cat in self.categories:
            cat_dir = os.path.join(root_dir, cat, split)
            if not os.path.exists(cat_dir):
                continue
            
            # 获取该类别下所有.xyz文件
            files = sorted([f for f in os.listdir(cat_dir) if f.endswith('.xyz')])
            
            if single_view:
                # 只保留每个物体的第一个视角
                unique_models = set('_'.join(f.split('_')[:-1]) for f in files)
                files = [next(f for f in files if f.startswith(model)) for model in unique_models]
            
            cat_idx = self.cat_to_idx[cat]
            self.data_paths.extend([os.path.join(cat_dir, f) for f in files])
            self.labels.extend([cat_idx] * len(files))

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        point_cloud = np.loadtxt(self.data_paths[idx]).astype(np.float32)  # 使用float32减少内存使用
        label = self.labels[idx]
        
        # 转换为torch张量
        point_cloud = torch.from_numpy(point_cloud)  # 使用from_numpy而不是FloatTensor
        label = torch.tensor(label, dtype=torch.long)
        
        return point_cloud, label

def get_data_loaders(root_dir='./modelnetdata/modelnetdata', batch_size=32, num_workers=None):
    """
    获取ModelNet的训练和测试数据加载器
    Args:
        root_dir: 数据集根目录
        batch_size: batch大小
        num_workers: 数据加载的进程数，如果为None则自动设置
    """
    # 自动设置num_workers
    if num_workers is None:
        num_workers = 0 if os.name == 'nt' else 4  # Windows上设为0，Linux上设为4
    
    # 检测是否可以使用GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pin_memory = device.type == 'cuda'  # 只在使用GPU时启用pin_memory
    
    # 创建数据集
    train_dataset = ModelNetDataset(root_dir, split='train')
    test_dataset = ModelNetDataset(root_dir, split='test')
    
    print(f"训练集大小: {len(train_dataset)}")
    print(f"测试集大小: {len(test_dataset)}")
    print(f"类别数量: {len(train_dataset.categories)}")
    print(f"使用设备: {device}")
    print(f"数据加载进程数: {num_workers}")
    
    # 创建数据加载器
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=True
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=False
    )
    
    return train_loader, test_loader, device

if __name__ == "__main__":
    # 获取数据加载器和设备信息
    train_loader, test_loader, device = get_data_loaders(batch_size=32)
    
    # 检查一个batch的数据
    for points, labels in train_loader:
        # 移动数据到对应设备
        points = points.to(device)
        labels = labels.to(device)
        
        print(f"Batch points shape: {points.shape}")
        print(f"Batch labels shape: {labels.shape}")
        break

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import DynamicEdgeConv, global_max_pool
from torch_geometric.data import Data, Batch

class DynamicEdgeConvNet(nn.Module):
    def __init__(self, in_channels=3, k=20, num_classes=40):
        super().__init__()
        
        # 动态图卷积层
        self.conv1 = DynamicEdgeConv(self._make_mlp(2*in_channels, [64, 64]), k=k, aggr='max')
        self.conv2 = DynamicEdgeConv(self._make_mlp(2*64, [64, 64]), k=k, aggr='max')
        self.conv3 = DynamicEdgeConv(self._make_mlp(2*64, [64, 64]), k=k, aggr='max')
        
        # 分类MLP
        self.mlp = nn.Sequential(
            nn.Linear(64*3, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def _make_mlp(self, in_channels, channels):
        layers = []
        for channel in channels:
            layers.extend([
                nn.Linear(in_channels, channel),
                nn.BatchNorm1d(channel),
                nn.ReLU(inplace=True)
            ])
            in_channels = channel
        return nn.Sequential(*layers)
    
    def forward(self, x, batch=None):
        # 如果输入是普通张量，转换为PyG的Data对象
        if not isinstance(x, Data) and not isinstance(x, Batch):
            batch = batch if batch is not None else torch.zeros(x.size(0), dtype=torch.long)
            x = Data(x=x, batch=batch)
        
        # 特征提取
        x1 = self.conv1(x.x, batch)  # [N, 64]
        x2 = self.conv2(x1, batch)   # [N, 64]
        x3 = self.conv3(x2, batch)   # [N, 64]
        
        # 特征拼接和全局池化
        out = torch.cat([x1, x2, x3], dim=1)  # [N, 192]
        out = global_max_pool(out, batch)      # [B, 192]
        
        # 分类
        out = self.mlp(out)
        return out

def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for points, labels in train_loader:
        points, labels = points.to(device), labels.to(device)
        batch = torch.arange(points.size(0)).repeat_interleave(points.size(1)).to(device)
        points = points.view(-1, 3)  # [B*N, 3]
        
        optimizer.zero_grad()
        out = model(points, batch)
        loss = criterion(out, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = out.argmax(dim=1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(train_loader), correct / total

def test(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for points, labels in test_loader:
            points, labels = points.to(device), labels.to(device)
            batch = torch.arange(points.size(0)).repeat_interleave(points.size(1)).to(device)
            points = points.view(-1, 3)
            
            out = model(points, batch)
            loss = criterion(out, labels)
            
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += pred.eq(labels).sum().item()
            total += labels.size(0)
    
    return total_loss / len(test_loader), correct / total

## 训练

In [ ]:
import os
import torch
import torch_cluster
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm  
import numpy as np
from datetime import datetime

def train(config):
    # 创建保存目录
    save_dir = os.path.join('runs', datetime.now().strftime('%Y%m%d_%H%M%S'))
    os.makedirs(save_dir, exist_ok=True)
    
    # 获取数据加载器和设备
    train_loader, test_loader, device = get_data_loaders(
        root_dir=config['data_dir'],
        batch_size=config['batch_size'],
        num_workers=config['num_workers']
    )
    
    # 初始化模型
    model = DynamicEdgeConvNet(
        in_channels=3,
        k=config['k_neighbors'],
        num_classes=40
    ).to(device)
    
    # 定义优化器和损失函数
    optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])
    criterion = nn.CrossEntropyLoss()
    
    # 记录训练历史
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    # 记录最佳准确率
    best_acc = 0.0
    
    # 训练循环
    from tqdm.auto import tqdm  # 使用auto替代notebook
    progress_bar = tqdm(range(config['epochs']), desc='Training')
    
    for epoch in progress_bar:
        # 训练阶段
        train_loss, train_acc = train_epoch(
            model=model,
            train_loader=train_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device
        )
        
        # 测试阶段
        test_loss, test_acc = test(
            model=model,
            test_loader=test_loader,
            criterion=criterion,
            device=device
        )
        
        # 更新学习率
        scheduler.step()
        
        # 记录历史
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        # 更新进度条
        progress_bar.set_postfix({
            'train_loss': f'{train_loss:.4f}',
            'train_acc': f'{train_acc:.4f}',
            'test_loss': f'{test_loss:.4f}',
            'test_acc': f'{test_acc:.4f}'
        })
        
        # 保存最佳模型
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_acc': best_acc,
            }, os.path.join(save_dir, 'best_model.pth'))
    
    print(f'Training completed! Best test accuracy: {best_acc:.4f}')
    return history, best_acc

# 训练配置
config = {
    'data_dir': './modelnetdata/modelnetdata',
    'batch_size': 32,
    'num_workers': 0,  # Windows上建议设为0
    'k_neighbors': 20,
    'epochs': 200,
    'lr': 0.001,
    'weight_decay': 1e-4,
}

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# 开始训练
history, best_acc = train(config)



In [ ]:
# 绘制训练曲线
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# 损失曲线
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['test_loss'], label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()

# 准确率曲线
plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['test_acc'], label='Test Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()

plt.tight_layout()
plt.show()